<a href="https://colab.research.google.com/github/worldwidekatie/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

### Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.

In [0]:
df['SALE_PRICE']

0              0
1              0
2              0
3              0
4              0
          ...   
23035     510000
23036     355000
23037     375000
23038    1100000
23039     569202
Name: SALE_PRICE, Length: 23040, dtype: int64

In [0]:
df2 = df[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') & 
         (df['SALE_PRICE'] > 100000) & 
         (df['SALE_PRICE'] < 2000000)]
df2.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


### Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [0]:
print(df2['SALE_DATE'].min())
print(df2['SALE_DATE'].max())
df2['SALE_DATE'].describe()

01/01/2019
04/30/2019


count           3151
unique            91
top       01/31/2019
freq              78
Name: SALE_DATE, dtype: object

In [0]:
#Can't get this to work so I'm giving up on it and going with the easier way.
df2['SALE_DATE'] = pd.to_datetime(df2['SALE_DATE'], infer_datetime_format=True)
cutoff = pd.to_datetime('04/01/2019')
train = df[df.SALE_DATE < cutoff]
test = df[df.SALE_DATE > cutoff]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


TypeError: ignored

In [0]:
train = df[df.SALE_DATE < '04/01/2019']
test = df[df.SALE_DATE >= '04/01/2019']
train.shape, test.shape

((18167, 21), (4873, 21))

In [0]:
train = train.fillna(0)
test = test.fillna(0)

### Do one-hot encoding of categorical features.

In [0]:
train.describe(exclude = 'number')

In [0]:
print(df2['BLOCK'].value_counts())
print(df2['LOT'].value_counts())
print(df2['ZIP_CODE'].value_counts())
print(df2['RESIDENTIAL_UNITS'].value_counts())
print(df2['COMMERCIAL_UNITS'].value_counts())
print(df2['TOTAL_UNITS'].value_counts())
print(df2['GROSS_SQUARE_FEET'].value_counts())
print(df2['YEAR_BUILT'].value_counts())
print(df2['TAX_CLASS_AT_TIME_OF_SALE'].value_counts())

df2.describe()

In [0]:
target = 'SALE_PRICE'

#First I should take away the ones with a ridiculous amount of categories
high_cardinality = ['ADDRESS', 'APARTMENT_NUMBER','LAND_SQUARE_FEET', 'SALE_DATE', 'EASE-MENT']
features = train.columns.drop([target] + high_cardinality)
features

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'BUILDING_CLASS_AT_PRESENT',
       'ZIP_CODE', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE',
       'BUILDING_CLASS_AT_TIME_OF_SALE'],
      dtype='object')

In [0]:
X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

X_train.shape, X_test.shape

((18167, 15), (4873, 15))

In [0]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test) #And we don't use fit here so the model doesn't 
                                     #Learn from the test data
X_train.shape, X_test.shape #That's way too many columns but I took away the numeric
                            #columns that had thousands of categories so idk what's 
                            #happening RN.

((18167, 358), (4873, 358))

In [0]:
X_train.head()

### Do feature selection with `SelectKBest`

In [0]:
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(score_func=f_regression, k=50)

X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

X_train_selected.shape, X_test_selected.shape

((18167, 50), (4873, 50))

In [0]:
selected_mask = selector.get_support()
all_names = X_train.columns
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features Selected:')
for name in selected_names:
  print(name)

print('\n')
print('Features not selected:')
print('Too many Features Im not printing this')
#for name in unselected_names:
    #print(name)

Features Selected:
BOROUGH_1
BOROUGH_4
BUILDING_CLASS_CATEGORY_21 OFFICE BUILDINGS
BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS
BUILDING_CLASS_CATEGORY_02 TWO FAMILY DWELLINGS
BUILDING_CLASS_CATEGORY_32 HOSPITAL AND HEALTH FACILITIES
BUILDING_CLASS_CATEGORY_33 EDUCATIONAL FACILITIES
BUILDING_CLASS_CATEGORY_08 RENTALS - ELEVATOR APARTMENTS
BUILDING_CLASS_CATEGORY_22 STORE BUILDINGS
BUILDING_CLASS_CATEGORY_30 WAREHOUSES
BUILDING_CLASS_CATEGORY_25 LUXURY HOTELS
BUILDING_CLASS_CATEGORY_11A CONDO-RENTALS
BUILDING_CLASS_CATEGORY_28 COMMERCIAL CONDOS
TAX_CLASS_AT_PRESENT_4
TAX_CLASS_AT_PRESENT_1
BLOCK
BUILDING_CLASS_AT_PRESENT_D6
BUILDING_CLASS_AT_PRESENT_E1
BUILDING_CLASS_AT_PRESENT_D1
BUILDING_CLASS_AT_PRESENT_O4
BUILDING_CLASS_AT_PRESENT_E9
BUILDING_CLASS_AT_PRESENT_O6
BUILDING_CLASS_AT_PRESENT_H1
BUILDING_CLASS_AT_PRESENT_RR
BUILDING_CLASS_AT_PRESENT_D8
BUILDING_CLASS_AT_PRESENT_W8
BUILDING_CLASS_AT_PRESENT_R5
BUILDING_CLASS_AT_PRESENT_K3
BUILDING_CLASS_AT_PRESENT_I4
BUILDING_CLASS_AT_

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns)-308):
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test Mean Absolute Error: ${mae:,.0f} \n')

1 features
Test Mean Absolute Error: $1,781,542 

2 features
Test Mean Absolute Error: $1,781,576 

3 features
Test Mean Absolute Error: $1,966,740 

4 features
Test Mean Absolute Error: $1,877,914 

5 features
Test Mean Absolute Error: $1,877,914 

6 features
Test Mean Absolute Error: $1,813,726 

7 features
Test Mean Absolute Error: $1,813,726 

8 features
Test Mean Absolute Error: $1,804,004 

9 features
Test Mean Absolute Error: $1,808,980 

10 features
Test Mean Absolute Error: $1,808,551 

11 features
Test Mean Absolute Error: $1,779,940 

12 features
Test Mean Absolute Error: $1,786,961 

13 features
Test Mean Absolute Error: $1,877,944 

14 features
Test Mean Absolute Error: $1,877,944 

15 features
Test Mean Absolute Error: $1,878,474 

16 features
Test Mean Absolute Error: $1,879,076 

17 features
Test Mean Absolute Error: $1,879,076 

18 features
Test Mean Absolute Error: $1,879,257 

19 features
Test Mean Absolute Error: $1,880,043 

20 features
Test Mean Absolute Error: $1

###  Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do feature scaling beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's transform method with the test set)

In [98]:
from sklearn.linear_model import RidgeCV
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]
ridge = RidgeCV(alphas=alphas, normalize=True)
ridge.fit(X_train, y_train)
ridge.alpha_

1.0

In [100]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=1, normalize=True)
ridge.fit(X_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=True,
      random_state=None, solver='auto', tol=0.001)

### Get mean absolute error for the test set.

In [101]:
y_pred = ridge.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Test MAE:", mae)


Test MAE: 1802862.4559406277


# Stretch Goals

In [0]:
#....Not Today